In [ ]:
# @title Inspect the schema of esdemo-389207.retail_data.customers
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('esdemo-389207.retail_data.customers')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:
# @title Setup bigquery client and formatting
from google.cloud import bigquery
from google.colab import data_table

project = 'esdemo-389207' # Project ID
client = bigquery.Client(project=project)
data_table.enable_dataframe_formatter()
# @title Executes the query, using client from previous cell
sql_customer_country = '''# What are the distinct countries in the customer table?

SELECT
  DISTINCT country
FROM
  `esdemo-389207`.`retail_data`.`customers`;'''
query_customer_country = client.query(sql_customer_country)

# store the destination to reference in future cells
query_destination_customer_country = query_customer_country.destination.project + '.' + query_customer_country.destination.dataset_id + '.' + query_customer_country.destination.table_id

#  Render the query results
job_customer_country= client.get_job(query_customer_country.job_id)
df_customer_country = job_customer_country.to_dataframe()
df_customer_country


In [ ]:
# @title Inspect the schema of esdemo-389207.retail_data.orders_dp
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Fetch the table
table = client.get_table('esdemo-389207.retail_data.orders_dp')

# View table info
print("{} rows".format(table.num_rows))
for row in table.schema:
  print("Column {}: {}".format(row.name, row.field_type))


In [ ]:

# @title Executes the query, using client from previous cell
sql_shipping_method_denmark = '''# What are the counts of shipping methods in Denmark?

SELECT
  shipping_method,
  COUNT(*) AS count
FROM
  `esdemo-389207`.`retail_data`.`orders_dp` AS orders_dp
INNER JOIN
  `esdemo-389207`.`retail_data`.`customers` AS customers
ON
  orders_dp.customer_id = customers.customer_id
WHERE
  customers.country = 'Denmark'
GROUP BY
  shipping_method;'''
query_shipping_method_denmark = client.query(sql_shipping_method_denmark)

# store the destination to reference in future cells
query_destination_shipping_method_denmark = query_shipping_method_denmark.destination.project + '.' + query_shipping_method_denmark.destination.dataset_id + '.' + query_shipping_method_denmark.destination.table_id

#  Render the query results
job_shipping_method_denmark= client.get_job(query_shipping_method_denmark.job_id)
df_shipping_method_denmark = job_shipping_method_denmark.to_dataframe()
df_shipping_method_denmark


In [ ]:
# @title 

import altair as alt

chart1 = alt.Chart(data=df1, mark={
  "type": "bar",
  "tooltip": True
}).encode(
  x={
  "field": "shipping_method",
  "type": "nominal",
  "title": "Shipping Method",
  "axis": {
    "labelOverlap": True
  },
  "sort": {}
},
  y={
  "field": "count",
  "type": "quantitative",
  "title": "Count",
  "axis": {
    "labelOverlap": True
  },
  "sort": {}
},
  
  
)
chart1


* The preferred Shipping Method is Pickup Point, representing 27.23% of total shipments.
	* 
* Express Shipping is the least utilized method at 21.76% of total shipments.
	* 
* On average, each Shipping Method handles approximately 443.5 shipments.
	* 

In [ ]:

# @title Executes the query, using client from previous cell
sql_SQL = '''-- prompts:
-- 1. Join these data sources
-- 2. Sales per day including customer city and country, include payment method, shipping method, status and preferred payment method 

SELECT
  customers.city,
  customers.country,
  orders_dp.order_date,
  orders_dp.payment_method,
  orders_dp.shipping_method,
  orders_dp.status,
  customers.preferred_payment_method,
  SUM(orders_dp.total_amount) AS total_sales
FROM
  `esdemo-389207`.`retail_data`.`customers` AS customers
INNER JOIN
  `esdemo-389207`.`retail_data`.`orders_dp` AS orders_dp
ON
  customers.customer_id = orders_dp.customer_id
GROUP BY
  1,
  2,
  3,
  4,
  5,
  6,
  7;'''
query_SQL = client.query(sql_SQL)

# store the destination to reference in future cells
query_destination_SQL = query_SQL.destination.project + '.' + query_SQL.destination.dataset_id + '.' + query_SQL.destination.table_id

#  Render the query results
job_SQL= client.get_job(query_SQL.job_id)
df_SQL = job_SQL.to_dataframe()
df_SQL


In [ ]:
# @title 

import altair as alt

chart2 = alt.Chart(data=df2, mark={
  "type": "bar",
  "tooltip": True
}).encode(
  x={
  "field": "city",
  "type": "nominal",
  "title": "City",
  "axis": {
    "labelOverlap": True
  },
  "sort": {}
},
  y={
  "field": "total_sales",
  "type": "quantitative",
  "title": "Total Sales",
  "axis": {
    "labelOverlap": True
  },
  "sort": {}
},
  color={
  "field": "payment_method",
  "type": "nominal",
  "title": "Payment Method"
},
  
)
chart2


* Uppsala accounts for the largest portion of total sales at 8.19%, with Reykjavik representing the smallest portion at only 0.19%.
	* 
* Debit cards represent the most popular payment method at 18.54% of total sales, while Vipps constitutes 15.45% of sales.
	* 
* Uppsala shows a strong preference for Vipps, with this payment method making up 21.3% of the city's sales.  This contrasts with the national trend where Debit Card is the leading payment method.
	* 
* Interestingly, while representing a small percentage of overall sales, Hafnarfjörður sees extremely low sales when using Debit Cards, suggesting a potential area for further investigation.
	* 